In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')
semi_df = semi_df[semi_df['Alignment'] != 'neutral']
semi_df = semi_df.replace(['good', 'bad'], [0, 1])

In [8]:
semi_df['Intelligence (%)'] = semi_df['Intelligence']/semi_df['Total']
semi_df['Power (%)'] = semi_df['Power']/semi_df['Total']
semi_df['Strength (%)'] = semi_df['Strength']/semi_df['Total']
semi_df['Combat (%)'] = semi_df['Combat']/semi_df['Total']

In [9]:
# Existem muitos personagens nesses dados cuja soma Total é composta de 
# Intelligence  Strength    Speed    Durability   Power   Combat  Total
#     1            1          1         1           0       1       5
semi_df = semi_df[semi_df['Total'] != 5]

In [10]:
import statsmodels.api as sm

# model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Strength (%)', 
#                                                        'Power', 'Power (%)', 'Alignment']])
model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power', 'Alignment']])
result = model.fit()
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.243
Model:                            OLS   Adj. R-squared (uncentered):              0.223
Method:                 Least Squares   F-statistic:                              12.42
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                    8.64e-09
Time:                        22:52:47   Log-Likelihood:                          525.97
No. Observations:                 159   AIC:                                     -1044.
Df Residuals:                     155   BIC:                                     -1032.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0001   2.95e-05      3.520      0.001    4.55e-05       0.000
Strength       1.83e-05   2.42e-05      0.756      0.451   -2.95e-05    6.61e-05
Power        -3.265e-05   3.04e-05     -1.073      0.285   -9.28e-05    2.75e-05
Alignment       -0.0044      0.002     -2.634      0.009      -0.008      -0.001
==============================================================================
Omnibus:                      186.455   Durbin-Watson:                   0.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5119.056
Skew:                           4.623   Prob(JB):                         0.00
Kurtosis:                      29.215   Cond. No.                         237.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
print(f"Coef:\n{result.params}\n")
print(f"std err:\n{result.bse}\n")
print(f"p-values:\n{result.pvalues}\n")

Coef:
Intelligence    0.000104
Strength        0.000018
Power          -0.000033
Alignment      -0.004397
dtype: float64

std err:
Intelligence    0.000029
Strength        0.000024
Power           0.000030
Alignment       0.001669
dtype: float64

p-values:
Intelligence    0.000567
Strength        0.450534
Power           0.284987
Alignment       0.009299
dtype: float64



In [12]:
# g = fm.load('marvelWeights.gml')
# g.label_nodes('name')
# g.set_all_nodes(size=15, labpos='hover')
# g.set_all_edges(color=(0, 0, 0, 0.5))

# intelligence = {}

# for i in g.nodes:
#     intelligence[i] = float(g.nodes[i]['intelligence'])

# g.scale_nodes_size(intelligence)
# g.draw()

In [13]:
semi_df.sort_values(by='Intermediation (r)', ascending=True).head(50)

,id,Name,ConnAmnt,Intermediation (r),Alignment,Intelligence,Strength,Speed,Durability,Power,Combat,Total,Intelligence (%),Power (%),Strength (%),Combat (%)
193,1011349,Blackout,6.0,0.000000,1,63,32,45,80,38,65,323,0.195046,0.117647,0.099071,0.201238
161,1009721,John Wraith,2.0,0.000000,0,75,12,35,28,65,80,295,0.254237,0.220339,0.040678,0.271186
182,1011088,Morlun,7.0,0.000000,1,63,60,35,42,59,56,315,0.200000,0.187302,0.190476,0.177778
183,1010764,Stardust,8.0,0.000000,0,88,85,100,110,100,85,568,0.154930,0.176056,0.149648,0.149648
190,1011071,Demogoblin,14.0,0.000002,1,50,48,42,35,54,60,289,0.173010,0.186851,0.166090,0.207612
141,1011138,Beyonder,23.0,0.000002,0,88,100,23,100,100,56,467,0.188437,0.214133,0.214133,0.119914
168,1011370,Molten Man,8.0,0.000005,1,50,73,23,84,53,56,339,0.147493,0.156342,0.215339,0.165192
153,1011293,Ink,20.0,0.000007,0,38,28,17,40,66,50,239,0.158996,0.276151,0.117155,0.209205
187,1009587,Shriek,22.0,0.000007,0,50,10,23,28,45,42,198,0.252525,0.227273,0.050505,0.212121
192,1011073,Bloodaxe,11.0,0.000009,1,63,80,33,80,84,84,424,0.148585,0.198113,0.188679,0.198113
